## Discrete Fourier Transform

In [48]:
function dft(x)
    N = length(x)
    X = complex(zeros(1, N))
    for k = 1:N
        for n = 1:N
            X[k] += x[n] * exp(-2*pi*im*(k-1)*(n-1)/N)
        end
    end
    return X'
end

dft (generic function with 1 method)

In [49]:
dft([1, 2, 3, 4])

4×1 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
                10.0 - 0.0im
 -2.0000000000000004 - 1.9999999999999996im
                -2.0 + 9.797174393178826e-16im
 -1.9999999999999982 + 2.000000000000001im

## Fast Fourier Transform

In [50]:
function fft(x)
    N = length(x)
    if N <= 1
        return x
    end
    
    X_even = fft(x[1:2:end])
    X_odd = fft(x[2:2:end])

    terms = @. exp(-2 * pi * im * (0:(N/2 - 1)) / N)

    X = [X_even .+ terms .* X_odd; X_even .- terms .* X_odd]
    
    return X
end

fft (generic function with 2 methods)

In [51]:
fft([1, 2, 3, 4])

4-element Vector{ComplexF64}:
                10.0 - 0.0im
                -2.0 + 2.0im
                -2.0 + 0.0im
 -1.9999999999999998 - 2.0im

## Benchmarking

In [52]:
using StatsBase
using BenchmarkTools

testVals = 100*rand(Complex{Float64}, 10000);

In [53]:
@benchmark dft(x) setup = (x = StatsBase.sample(testVals, 256, replace=true))

BenchmarkTools.Trial: 4269 samples with 1 evaluation.
 Range (min … max):  1.056 ms …   8.102 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.131 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.158 ms ± 191.718 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▇█▄▃▂   ▃▅▁▁▁                                               
  ███████▇▇█████▆▄▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂ ▃
  1.06 ms         Histogram: frequency by time        1.63 ms <

 Memory estimate: 6.27 KiB, allocs estimate: 3.

In [54]:
@benchmark fft(x) setup = (x = StatsBase.sample(testVals, 256, replace=true))

BenchmarkTools.Trial: 6469 samples with 1 evaluation.
 Range (min … max):  592.900 μs …   8.931 ms  ┊ GC (min … max): 0.00% … 91.92%
 Time  (median):     691.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   760.545 μs ± 496.213 μs  ┊ GC (mean ± σ):  6.51% ±  8.96%

  ██▆▄▃▂▂▁▁▁                                                    ▁
  ██████████▇▆▆▅▄▃▁▃▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▄▅ █
  593 μs        Histogram: log(frequency) by time       4.51 ms <

 Memory estimate: 590.16 KiB, allocs estimate: 8432.

### Comparing median times

In [56]:
691e-6 / 1.131e-3

0.6109637488947833

Fast Fourier Transform is consistently faster, taking approximately 39% lesser time, albeit using higher processing power.